# ipyleaflet

In [1]:
from ipyleaflet import Map, basemaps

In [2]:
center = [52.5162, 13.3777]
zoom = 6

In [3]:
Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)

Map(center=[52.5162, 13.3777], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [4]:
Map(basemap=basemaps.Strava.All, center=center, zoom=zoom)

Map(center=[52.5162, 13.3777], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [5]:
from ipyleaflet import WidgetControl
from ipywidgets import IntSlider, jslink


m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center)
zoom_slider = IntSlider(description="Zoom level:", min=0, max=19, value=9)
jslink((zoom_slider, "value"), (m, "zoom"))
widget_control1 = WidgetControl(widget=zoom_slider, position="topright")
m.add_control(widget_control1)

m

Map(center=[52.5162, 13.3777], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [6]:
from ipywidgets import Label

label = Label()
display(label)

def handle_interaction(**kwargs):
    if kwargs.get('type') == 'mousemove':
        label.value = str(kwargs.get('coordinates'))
m.on_interaction(handle_interaction)

m

Label(value='')

Map(bottom=43188.0, center=[52.5162, 13.3777], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoo…

In [7]:
import json
import os

import pandas as pd
import requests

from geojson import GeoJSON
from ipyleaflet import LayersControl


if not os.path.exists("europe_110.geo.json"):
    url = "https://github.com/jupyter-widgets/ipyleaflet/raw/master/examples/europe_110.geo.json"
    r = requests.get(url)
    with open("europe_110.geo.json", "w") as f:
        f.write(r.content.decode("utf-8"))
with open("europe_110.geo.json", "r") as f:
    data = json.load(f)

geo_json = GeoJSON(
    data=data,
    style={"opacity": 1, "dashArray": "9", "fillOpacity": 0.1, "weight": 1},
    hover_style={"color": "white", "dashArray": "0", "fillOpacity": 0.5},
    style_callback={"color": "black", "fillColor": "grey"},
)
m.add_layer(geo_json)
m.add_control(LayersControl())

m

Map(bottom=43188.0, center=[52.5162, 13.3777], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoo…

In [8]:
import ipywidgets


ipywidgets.HBox([m, Map(center=[52.5162, 13.3777], zoom=5)])

## WMS-Layer

Mit ipyleaflet lassen sich auch eigene [WMS](https://de.wikipedia.org/wiki/Web_Map_Service)-Layer definiern. Üblicherweise werden Optionen wie `layers`, `format` und `transparent` in der Request-URL übergeben. Wenn ihr  zusätzliche Parameter benötigt, könnt ihr diese jedoch auch in eigenen WMSLayer-Klassen definieren. Im folgenden wird beispielsweise ein Zeitparameter hinzugefügt, indem ein benutzerdefinierter `TimeWMSLayer` definiert wird.

### Beispiel für einen eigenen WMS-Layer

In [9]:
from ipyleaflet import Map, WMSLayer, basemaps
from traitlets import Unicode


class TimeWMSLayer(WMSLayer):
    time = Unicode("").tag(sync=True, o=True)


time_wms = TimeWMSLayer(
    url="https://mesonet.agron.iastate.edu/cgi-bin/wms/nexrad/n0r-t.cgi?",
    layers="nexrad-n0r-wmst",
    time="2005-08-29T13:00:00Z",
    format="image/png",
    transparent=True,
    attribution="Weather data © 2012 IEM Nexrad",
)

m = Map(basemap=basemaps.CartoDB.Positron, center=(30.661, -88.645), zoom=5)

m.add_layer(time_wms)

m

Map(center=[30.661, -88.645], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

Da es sich um ein Widget handelt, könnt ihr den WMS-Parameter `time_wms.time` auch einfach ändern:

In [10]:
time_wms.time = "2005-08-29T19:00"

Oder ihr könnt einen Slider mit passenden Zeitintervallen definieren:

In [11]:
from ipywidgets import SelectionSlider


time_options = [
    "13:00",
    "13:30",
    "14:00",
    "14:30",
    "15:00",
    "15:30",
    "16:00",
    "16:30",
]

slider = SelectionSlider(description="Time:", options=time_options)


def update_wms(change):
    time_wms.time = "2005-08-29T{}".format(slider.value)


slider.observe(update_wms, "value")

slider

SelectionSlider(description='Time:', options=('13:00', '13:30', '14:00', '14:30', '15:00', '15:30', '16:00', '…

### WMS-Layer-Attribute

| Attribute | Standardwert | Kommentar |
| :-------- | :----------- | :-------- |
| `url` | `"https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png"` |  |
| `min_zoom` | `0` |  |
| `max_zoom` | `18` |  |
| `tile_size` | `256` |  |
| `attribution` | `"Map data (c) <a href='https://openstreetmap.org'>OpenStreetMap</a> contributors"` |  |
| `detect_retina` | `False` |  |
| `opacity` | `1.0` |  |
| `visible` | `True` |  |
| `service` | `"WMS"` |  |
| `request` | `"GetMap"` |  |
| `layers` | `""` | Kommagetrennte Liste der anzuzeigenden WMS-Ebenen |
| `styles` | `""` | Kommagetrennte Liste der WMS-Stile |
| `format` | `"image/jpeg"` | Für Ebenen mit Transparenz solltet ihr `"image/png"` verwenden |
| `transparent` | `False` |  |
| `version` | `"1.1.1"` | Version des zu verwendenden WMS-Dienstes |
| `crs` | `""` |  |
